<a href="https://colab.research.google.com/github/prof-rossetti/intro-to-python/blob/main/units/msfo-833/Class_3_Exercises_(MSFO_833_Spring_2023)_SOLUTIONS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> NOTE: this notebook adapted from material created by Prof. Ram Yamarthy (2021).

## Setup

### Imports

In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd, YearEnd

import plotly.express as px
import matplotlib.pyplot as plt

import statsmodels.api as sm

# https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html
# https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html
# from statsmodels.api import OLS, add_constant

# https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.acf.html
from statsmodels.tsa.stattools import acf

# https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMA.html
from statsmodels.tsa.arima.model import ARIMA

### Ignore Warnings

In [ ]:
# https://stackoverflow.com/a/9031848/670433
import warnings
warnings.filterwarnings('ignore')

## Part 1 -- Seasonality and Trends in Time Series


#### Example 1 -- US Population

##### Load & Plot Data

In [ ]:
filename = "uspopulation.xlsx"
uspop = pd.read_excel(f"https://github.com/s2t2/msfo-833-prep/blob/main/data/{filename}?raw=true")
uspop.head()

,Month,Year,US_Population
0,1,1959,175818
1,2,1959,176044
2,3,1959,176274
3,4,1959,176503
4,5,1959,176723


In [ ]:
uspop.dtypes

Month            int64
Year             int64
US_Population    int64
dtype: object

In [ ]:
# sort for good measure
uspop.sort_values(by=["Year", "Month"], ascending=True, inplace=True)
uspop.head()

,Month,Year,US_Population
0,1,1959,175818
1,2,1959,176044
2,3,1959,176274
3,4,1959,176503
4,5,1959,176723


In [ ]:
px.line(y=uspop["US_Population"], title="US Population over Time", 
        labels={"y":"US Population (thousands)"},
        height=300
    )

In [ ]:
uspop["Year_Month"] = uspop["Year"].astype(str) + "-" + uspop["Month"].astype(str)  

#uspop["Date"] = pd.to_datetime(uspop["Year_Month"], format="%Y-%m")
uspop["Date"] = pd.to_datetime(uspop["Year_Month"], format="%Y-%m") + MonthEnd(0)

uspop.index = uspop["Date"]
uspop.head()

,Month,Year,US_Population,Year_Month,Date
Date,,,,,
1959-01-31,1,1959,175818,1959-1,1959-01-31
1959-02-28,2,1959,176044,1959-2,1959-02-28
1959-03-31,3,1959,176274,1959-3,1959-03-31
1959-04-30,4,1959,176503,1959-4,1959-04-30
1959-05-31,5,1959,176723,1959-5,1959-05-31


In [ ]:
px.line(uspop, x="Date", y="US_Population",  title="US Population over Time", 
        labels={"US_Population":"US Population (thousands)"},
        #height=300
    )

In [ ]:
px.scatter(uspop, x="Date", y="US_Population", title="US Population over time, vs linear trend",
           trendline="ols", trendline_color_override="red",
            #height=300
)

looks like a possible linear trend. let's perform a more formal regression analysis...

##### Estimate Trend (Linear Regression)

In [ ]:
# https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html
# https://www.statology.org/simple-linear-regression-in-python/
# ... ordinary least squares (OLS) regression model

y = uspop["US_Population"] 

time_range = range(1, len(uspop) + 1) # using a time series of integers as our x variable
x = sm.add_constant(time_range) # adding in a column of constants, as per the OLS docs
x = pd.DataFrame(x, columns=["const", "time"], index=y.index) # optionally naming the columns, specify index to avoid training error later

model = sm.OLS(y, x, missing="drop")
print(type(model))

results = model.fit()
print(type(results))

print(results.summary())

<class 'statsmodels.regression.linear_model.OLS'>
<class 'statsmodels.regression.linear_model.RegressionResultsWrapper'>
                            OLS Regression Results                            
Dep. Variable:          US_Population   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 2.310e+05
Date:                Thu, 19 Jan 2023   Prob (F-statistic):               0.00
Time:                        03:23:20   Log-Likelihood:                -6853.7
No. Observations:                 739   AIC:                         1.371e+04
Df Residuals:                     737   BIC:                         1.372e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------

After fitting / training, we have access to params, fitted values (i.e. predictions), and the prediction residuals (i.e. errors).


In [ ]:
print(results.params)
print("-----------")
print(f"y = {results.params['time']}x + {results.params['const']}")

const    174320.572450
time        214.089405
dtype: float64
-----------
y = 214.08940481786794x + 174320.57245013583


y = 214.09x + 174320.57

population = (214.09 * time_step) + 174300

pop at the earliest time period around 1960 i.e. intercept is around 175,000 K

##### Plot Trend vs. Actual Data, Residual

In [ ]:
# predicton results obtained after model has been trained:
uspop["OLS_Pred"] = results.fittedvalues
uspop["OLS_Resid"] = results.resid

# FYI: residuals should be equivalent to:
#uspop["OLS_Resid"] = uspop["US_Population"] - uspop["Pred_OLS"]

In [ ]:
px.line(uspop, x="Date", y=["US_Population", "OLS_Pred"], title="US Population vs prediction (Statsmodels OLS)")

In which years is the actual population above / below trend?

In [ ]:
px.line(uspop, x="Date", y="OLS_Resid", title="US Population vs trend residuals (Statsmodels OLS)")

In [ ]:
round(uspop["OLS_Resid"].mean())

0

##### Linear Regression (Sklearn Alternative)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
from sklearn.linear_model import LinearRegression
import numpy as np

lr = LinearRegression()

y = uspop["US_Population"] 

# represent sorted time values as a sequence of integers:
x = range(1, len(uspop) + 1) 
# only because in this particular example our x dataset contains a single column,
# ... we reshape to overcome error during training: "Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample"
# ... however in most cases the x dataset will have multiple columns and this step wouldn't be necessary:
x = np.array(x).reshape(-1,1)

# train the model
lr.fit(x, y)

print("COEFS:", lr.coef_)
print("INTERCEPT:", lr.intercept_)
print(f"y = {round(lr.coef_[0], 2)}x + {round(lr.intercept_, 2)}")

COEFS: [214.08940482]
INTERCEPT: 174320.57245013583
y = 214.09x + 174320.57


In [ ]:
# use the model to make prediction(s):
y_pred = lr.predict(x)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# evaluate predictions using various regression metrics:
r2 = r2_score(y, y_pred)
print("R SQUARED:", r2)

mse = mean_squared_error(y, y_pred)
print("MEAN SQUARED ERROR:", mse)

mae = mean_absolute_error(y, y_pred)
print("MEAN ABSOLUTE ERROR:", mae)

R SQUARED: 0.9968199397438178
MEAN SQUARED ERROR: 6654505.946520969
MEAN ABSOLUTE ERROR: 2245.7122314430785


In [ ]:
chart_df = uspop.copy()
chart_df["Prediction"] = y_pred
px.line(chart_df, x="Date", y=["US_Population", "Prediction"], title="US Population vs Prediction (SKlearn Linear Regression)")

In [ ]:
resid = y - y_pred

px.line(y=resid)

In [ ]:
round(resid.mean())

0

#### Example 2 -- US Employment Data, BLS


##### Load & Plot Data

In [ ]:
filename = "nsa_employ.xlsx"
employ = pd.read_excel(f"https://github.com/s2t2/msfo-833-prep/blob/main/data/{filename}?raw=true")
employ.head()

,Year,Month,Label,Quarter,Employment
0,1939,1,1939 Jan,1,29296
1,1939,2,1939 Feb,1,29394
2,1939,3,1939 Mar,1,29804
3,1939,4,1939 Apr,2,29786
4,1939,5,1939 May,2,30145


In [ ]:
#employ["Month"] = employ["Month"].astype(str).str.zfill(2)
#employ["Year_Month"] = employ["Year"].astype(str) + "-" + employ["Month"].astype(str)  
#employ["Date"] = pd.to_datetime(employ["Year_Month"], format="%Y-%m")
# alternative:

employ["Year_Month"] = employ["Year"].astype(str) + "-" + employ["Month"].astype(str)  
#employ["Date"] = pd.to_datetime(employ["Year_Month"], format="%Y-%m")
employ["Date"] = pd.to_datetime(employ["Year_Month"], format="%Y-%m") + MonthEnd(0)

employ

,Year,Month,Label,Quarter,Employment,Year_Month,Date
0,1939,1,1939 Jan,1,29296,1939-1,1939-01-31
1,1939,2,1939 Feb,1,29394,1939-2,1939-02-28
2,1939,3,1939 Mar,1,29804,1939-3,1939-03-31
3,1939,4,1939 Apr,2,29786,1939-4,1939-04-30
4,1939,5,1939 May,2,30145,1939-5,1939-05-31
...,...,...,...,...,...,...,...
975,2020,4,2020 Apr,2,130317,2020-4,2020-04-30
976,2020,5,2020 May,2,133432,2020-5,2020-05-31
977,2020,6,2020 Jun,2,138502,2020-6,2020-06-30
978,2020,7,2020 Jul,3,139063,2020-7,2020-07-31


In [ ]:
px.line(employ, x="Date", y="Employment", labels={"Employment": "Employment (in thousands)"},
        title="US Employment by month"
)

##### Observe Seasonality Through a Random Sample

In [ ]:
px.line(employ[employ["Year"].between(1970, 1980)], 
        x="Date", y="Employment", labels={"Employment": "Employment (in thousands)"},
        title="US Employment by month (selected years)"

)

##### Estimate Linear Trend

In [ ]:
y = employ["Employment"]

x = range(1, len(employ) + 1)
x = sm.add_constant(x)
x = pd.DataFrame(x, columns=["const", "time"])

model = sm.OLS(y, x)
print(type(model))

results = model.fit()
print(type(results))
print(results.summary())

<class 'statsmodels.regression.linear_model.OLS'>
<class 'statsmodels.regression.linear_model.RegressionResultsWrapper'>
                            OLS Regression Results                            
Dep. Variable:             Employment   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.982
Method:                 Least Squares   F-statistic:                 5.383e+04
Date:                Thu, 19 Jan 2023   Prob (F-statistic):               0.00
Time:                        03:23:26   Log-Likelihood:                -9715.2
No. Observations:                 980   AIC:                         1.943e+04
Df Residuals:                     978   BIC:                         1.944e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------

In [ ]:
print(results.rsquared)
print("------------")
print(results.params)

0.9821569717425276
------------
const    26598.425964
time       128.185926
dtype: float64


##### Plot Trend Line and Residual

In [ ]:
employ["OLS_Pred"] = results.fittedvalues
employ["OLS_Resid"] = results.resid 

In [ ]:
fig = px.line(employ, x="Date", y=["Employment", "OLS_Pred"], title="Employment vs trend", height=300)
fig.show()

fig = px.line(employ, x="Date", y="OLS_Resid", title="Employment vs trend (residuals)", height=300)
fig.show()

##### Observe Seasonality through Periodic Means

In [ ]:
print("***************")
print("Quarterly Means")
print("***************")
print(employ.groupby("Quarter")["OLS_Resid"].mean())

print(' ')
print("***************")
print("Monthly Means")
print("***************")
print(employ.groupby("Month")["OLS_Resid"].mean())


***************
Quarterly Means
***************
Quarter
1   -971.771456
2    113.626052
3     42.578019
4    825.811337
Name: OLS_Resid, dtype: float64
 
***************
Monthly Means
***************
Month
1    -1217.581466
2    -1049.499099
3     -648.233805
4     -368.566072
5      111.833369
6      597.610858
7     -230.465312
8     -100.968310
9      464.310760
10     755.235946
11     834.309280
12     887.888786
Name: OLS_Resid, dtype: float64


##### De-Seasonalizing Data Using a Regression

In [ ]:
x = employ["Month"]
#print(x.iloc[0:5])

# https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
# "one hot encode" the monthly values:
x = pd.get_dummies(x)
x.columns=["Jan", "Feb", "Mar", "Apr",
                "May", "Jun", "Jul", "Aug",
                "Sep", "Oct", "Nov", "Dec"]
x

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
975,0,0,0,1,0,0,0,0,0,0,0,0
976,0,0,0,0,1,0,0,0,0,0,0,0
977,0,0,0,0,0,1,0,0,0,0,0,0
978,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
y = employ["OLS_Resid"] # can we predict the residual (i.e. degree to which we will be over or under trend)?

monthly_ols = sm.OLS(y, x)
print(type(monthly_ols))

monthly_results = monthly_ols.fit()
print(type(monthly_results))

print(monthly_results.summary())


<class 'statsmodels.regression.linear_model.OLS'>
<class 'statsmodels.regression.linear_model.RegressionResultsWrapper'>
                            OLS Regression Results                            
Dep. Variable:              OLS_Resid   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.830
Date:                Thu, 19 Jan 2023   Prob (F-statistic):             0.0452
Time:                        03:23:28   Log-Likelihood:                -9705.1
No. Observations:                 980   AIC:                         1.943e+04
Df Residuals:                     968   BIC:                         1.949e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------

In [ ]:
employ["Monthly_OLS_Prediction"] = monthly_results.fittedvalues
employ["Monthly_OLS_Residual"] = monthly_results.resid

In [ ]:
height = 400

fig = px.line(employ, x="Date", y=["Employment", "OLS_Pred"], title="Employment vs trend", height=height)
fig.show()

fig = px.line(employ, x="Date", y="Monthly_OLS_Prediction", title="Employment vs seasonal trend", height=height)
fig.show()

fig = px.line(employ, x="Date", y="Monthly_OLS_Residual", title="Employment vs seasonal trend (residual)", height=height)
fig.show()

## Part 2 -- Estimating Autocorrelation


#### Example 1 -- Using Random Data

In [ ]:
x_rand = np.random.normal(0, 1, 1000)
print(type(x_rand))
print(x_rand.shape) # a one dimensional array of 1000 numbers

<class 'numpy.ndarray'>
(1000,)


In [ ]:

# https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.acf.html

from statsmodels.tsa.stattools import acf

n_lags = 10

acf_rand = acf(x_rand, nlags=n_lags, fft=False)
print(type(acf_rand)) #> numpy.ndarray
print(len(acf_rand))
print(acf_rand)

<class 'numpy.ndarray'>
11
[ 1.         -0.02864173  0.06454761  0.03696673 -0.02091214 -0.00215171
 -0.00406641 -0.03419017 -0.02612092  0.02450003  0.01338835]


In [ ]:
px.line(y=acf_rand, markers=["o"], 
        title="Auto-correlation of a time series of random numbers",
        labels={"x": "Number of Lags", "y":"Auto-correlation"}
)

In this randomly generated dataset, we see the current value has around no correlation with the following value(s). 

#### Example 2 -- Baseball Win-Loss Percentages

##### Load and Plot Data

In [ ]:
# hosted data:
filename = "bball_data.xlsx"
baseball_file_url = f"https://github.com/s2t2/msfo-833-prep/blob/main/data/{filename}?raw=true"

df_nyy = pd.read_excel(baseball_file_url, sheet_name="ny_yankees")
df_bos = pd.read_excel(baseball_file_url, sheet_name="bo_redsox")
df_bal = pd.read_excel(baseball_file_url, sheet_name="balt_orioles")
df_tor = pd.read_excel(baseball_file_url, sheet_name="tor_blujays")

print(len(df_nyy), len(df_bal), len(df_bal), len(df_tor))

for df in [df_nyy, df_bos, df_bal, df_tor]:
    df.index = df["Year"]

df_nyy.head()

118 119 119 44


,Year,Tm,Lg,G,W,L,Ties,W-L%
Year,,,,,,,,
2020,2020,New York Yankees,AL East,60,33,27,0,0.550
2019,2019,New York Yankees,AL East,162,103,59,0,0.636
2018,2018,New York Yankees,AL East,162,100,62,0,0.617
2017,2017,New York Yankees,AL East,162,91,71,0,0.562
2016,2016,New York Yankees,AL East,162,84,78,0,0.519


In [ ]:

#def summarize_baseball_dataset(df, team_name):
#    print("-------------------")
#    print("TEAM:", team_name)
#    print("YEARS:", df["Year"].min(), "to", df["Year"].max(), f"({len(df)} rows)")
#    print("AVG W/L:", round(df["W-L%"].mean(), 2))
#
#    fig = px.line(df, x="Year", y="W-L%", title=f"{team_name} Performance over Time", height=300)
#    fig.show()
#
#
#summarize_baseball_dataset(df_nyy, "NYY")
#
#summarize_baseball_dataset(df_bos, "BOS")
#
#summarize_baseball_dataset(df_bal, "BAL")
#
#summarize_baseball_dataset(df_tor, "TOR")

In [ ]:
# one dataset to rule them all, in this case because we are analyzing annual performance only:
perf = pd.DataFrame()
perf["NYY"] = df_nyy["W-L%"]
perf["BOS"] = df_bos["W-L%"]
perf["BAL"] = df_bal["W-L%"]
perf["TOR"] = df_tor["W-L%"]
perf

,NYY,BOS,BAL,TOR
Year,,,,
2020,0.550,0.400,0.417,0.533
2019,0.636,0.519,0.333,0.414
2018,0.617,0.667,0.290,0.451
2017,0.562,0.574,0.463,0.469
2016,0.519,0.574,0.549,0.549
...,...,...,...,...
1907,0.473,0.396,0.454,NaN
1906,0.596,0.318,0.510,NaN
1905,0.477,0.513,0.353,NaN


In [ ]:
perf.describe()

,NYY,BOS,BAL,TOR
count,118.000000,118.000000,118.000000,44.000000
mean,0.569475,0.516763,0.473398,0.493818
std,0.075258,0.082090,0.090071,0.070091
min,0.329000,0.279000,0.279000,0.327000
25%,0.520500,0.476500,0.409000,0.457000
50%,0.586000,0.526500,0.474500,0.512000
75%,0.623000,0.578500,0.543000,0.538500
max,0.714000,0.691000,0.673000,0.615000


In [ ]:
px.line(perf, y=["NYY", "BOS", "BAL", "TOR"], title="Baseball Team Annual Win Percentages",
        labels={"value": "Win Percentage", "variable": "Team"}
    )

In [ ]:
# just for fun, lets see a smoother trend, feel free to change the window

window = 10

perf[f"NYY_ma{window}"] = perf["NYY"].rolling(window=window).mean()
perf[f"BOS_ma{window}"] = perf["BOS"].rolling(window=window).mean()
perf[f"BAL_ma{window}"] = perf["BAL"].rolling(window=window).mean()
perf[f"TOR_ma{window}"] = perf["TOR"].rolling(window=window).mean()

px.line(perf, y=[f"NYY_ma{window}", f"BOS_ma{window}", f"BAL_ma{window}", f"TOR_ma{window}"], 
        title=f"Baseball Team Win Percentages ({window} Year Moving Avg)",
        labels={"value": "Win Percentage", "variable": "Team"}
    )

##### Auto-correlation

In [ ]:
# https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.acf.html
# auto-correlation function

#from statsmodels.tsa.stattools import acf

#def compute_baseball_acf(df, team_name, n_lags=10):
#    print("-------------------")
#    print("ACF FOR", team_name, "W/", n_lags, "LAGS...")
#
#    acf_results = acf(df["W-L%"], nlags=n_lags, fft=True) # FYI: setting fft param to suppress a warning
#
#    #periods = range(0, n_lags + 1)
#    #print(periods)
#
#    fig = px.line(y=acf_results, markers=["o"], height=300,
#            title=f"Auto-correlation of Annual Baseball Performance ({team_name})",
#            labels={"x": "Number of Lags", "y":"Auto-correlation"},   
#    )
#    fig.show()
#
#    return acf_results
#
#
#acf_results_nyy = compute_baseball_acf(yanks, "NYY")
#print(acf_results_nyy)
#
#
#acf_results_bos = compute_baseball_acf(redsox, "BOS")
#print(acf_results_bos)
#
#acf_results_balt = compute_baseball_acf(orioles, "BALT")
#print(acf_results_balt)
#
#acf_results_tor = compute_baseball_acf(bluejays, "TOR")
#print(acf_results_tor)

In [ ]:


def compute_baseball_acf(team_name, n_lags=10):

    print("-------------------")
    print("ACF FOR", team_name, "W/", n_lags, "LAGS...")

    acf_results = acf(perf[team_name], nlags=n_lags, fft=True, missing="drop") # FYI: setting fft param to suppress a warning
    print(acf_results)

    #periods = range(0, n_lags + 1)
    #print(periods)

    fig = px.line(y=acf_results, markers=["o"], height=300,
            title=f"Auto-correlation of Annual Baseball Performance ({team_name})",
            labels={"x": "Number of Lags", "y":"Auto-correlation"},   
    )
    fig.show()

    return acf_results

In [ ]:
acf_results_nyy = compute_baseball_acf("NYY")

acf_results_bos = compute_baseball_acf("BOS")

acf_results_bal = compute_baseball_acf("BAL")

acf_results_tor = compute_baseball_acf("TOR")

-------------------
ACF FOR NYY W/ 10 LAGS...
[1.         0.61148404 0.34890418 0.32078676 0.3459768  0.29498255
 0.2173049  0.18496153 0.11479746 0.07436017 0.06523038]


-------------------
ACF FOR BOS W/ 10 LAGS...
[ 1.          0.55413721  0.43996138  0.34047104  0.30106342  0.25413013
  0.14914321  0.09565376  0.02236638  0.00355596 -0.13700039]


-------------------
ACF FOR BAL W/ 10 LAGS...
[1.         0.66302003 0.51367353 0.37705783 0.30429253 0.20333063
 0.19140228 0.16515828 0.15976807 0.11604105 0.13702068]


-------------------
ACF FOR TOR W/ 10 LAGS...
[ 1.          0.57210965  0.36860427  0.06757568 -0.0046991  -0.01563252
 -0.16642747 -0.09578702 -0.158596   -0.09998359 -0.22702282]


In [ ]:
# same number and order of rows / n_lags for each column of results
chart_df = pd.DataFrame()
chart_df["NYY"] = acf_results_nyy
chart_df["BOS"] = acf_results_bos
chart_df["BAL"] = acf_results_bal
chart_df["TOR"] = acf_results_tor

px.line(chart_df, y=["NYY", "BOS", "BAL", "TOR"], markers="O",
        title="Auto-correlation of Annual Baseball Team Performance",
        labels={"variable": "Team", "value": "Auto-correlation", "index":"Number of lags"}
)

#### Example 3 -- Monthly Volatility Levels

##### Load and Plot Data

In [ ]:
# hosted data:
filename = "spdailydata.xlsx"
sp_daily = pd.read_excel(f"https://github.com/s2t2/msfo-833-prep/blob/main/data/{filename}?raw=true")
sp_daily.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0


In [ ]:
sp_daily.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

In [ ]:
print(sp_daily["Date"].min())
print(sp_daily["Date"].max())

print(sp_daily["Adj Close"].min())
print(sp_daily["Adj Close"].max())

1927-12-30 00:00:00
2020-10-30 00:00:00
4.4
3580.840088


In [ ]:
# sort before shift to be safe
sp_daily.sort_values(by=["Date"], ascending=True, inplace=True)

# these approaches are equivalent:
#sp_daily["Ret"] = sp_daily["Adj Close"] / sp_daily["Adj Close"].shift(1) - 1.0
sp_daily["Return"] = sp_daily["Adj Close"].pct_change(periods=1)

sp_daily.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Return
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0,NaN
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0,0.005663
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0,-0.002252
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0,-0.009594
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0,0.006268


In [ ]:
#sp_df[sp_daily["Ret"] != sp_daily["Return"]]

In [ ]:
px.line(sp_daily, x="Date", y="Return", title="Monthly Returns (S&P 500)")

##### Compute Monthly Volatilities Using Daily Data

In [ ]:
# already a datetime
#sp_daily["Date"] = pd.to_datetime(sp_daily["Date"])

sp_daily["Year"] = sp_daily["Date"].dt.year
sp_daily["Month"] = sp_daily["Date"].dt.month

sp_daily[["Date", "Year", "Month", "Return"]].head()

,Date,Year,Month,Return
0,1927-12-30,1927,12,NaN
1,1928-01-03,1928,1,0.005663
2,1928-01-04,1928,1,-0.002252
3,1928-01-05,1928,1,-0.009594
4,1928-01-06,1928,1,0.006268


In [ ]:
#sp_pivot = pd.pivot_table(sp_daily, index=["Year", "Month"],
#                          values=["Return"],
#                          aggfunc={"Return": np.std}
#                        )
#sp_pivot.columns = ["Volatility"]
#sp_pivot

In [ ]:
sp_monthly_volatility = sp_daily.groupby(["Year", "Month"])["Return"].std().reset_index()
print(type(sp_monthly_volatility))

# adjust col names:
sp_monthly_volatility.columns = ["Year", "Month", "Volatility"]

sp_monthly_volatility[["Year", "Month", "Volatility"]]

<class 'pandas.core.frame.DataFrame'>


,Year,Month,Volatility
0,1927,12,NaN
1,1928,1,0.007639
2,1928,2,0.006549
3,1928,3,0.007098
4,1928,4,0.009871
...,...,...,...
1110,2020,6,0.018499
1111,2020,7,0.008376
1112,2020,8,0.005112
1113,2020,9,0.015557


##### Auto-correlation of Monthly Volatility

In [ ]:
x = sp_monthly_volatility["Volatility"]

vol_acf = acf(x, nlags=12, missing="drop", fft=False)



#print(len(vol_acf)) #> 13 = 1 original + 12 lags
print(vol_acf)

px.line(y=vol_acf, height=500, markers="o",
        title="Auto-correlation of S&P Monthly Return Volatility",
        labels={"x": "Number of Lags", "y": "Auto-correlation"}
)

[1.         0.71098761 0.56945162 0.52903137 0.49229608 0.48663972
 0.49666699 0.49453475 0.48842952 0.44799854 0.43025416 0.4240305
 0.41790723]


## Part 3 -- ARMA Models

#### Example -- Autoregressive model for GDP Growth

##### Load and Plot Data

In [ ]:
filename = "gdp_data.xlsx"
gdp = pd.read_excel(f"https://github.com/s2t2/msfo-833-prep/blob/main/data/{filename}?raw=true")
gdp.head()

,Year,Quarter,Real GDP,Growth Rate
0,1947,1,2033.061,NaN
1,1947,2,2027.639,-0.002667
2,1947,3,2023.452,-0.002065
3,1947,4,2055.103,0.015642
4,1948,1,2086.017,0.015043


In [ ]:
gdp.dtypes

Year             int64
Quarter          int64
Real GDP       float64
Growth Rate    float64
dtype: object

In [ ]:
# pd.date_range("1947-01-01", "2020-09-30", freq = "q")

# alternative using month beginning:
# gdp["Month"] = gdp["Quarter"] * 3   # Q1:3, Q2:6, Q3: 9, Q4: 12 
# gdp["Date"] = gdp["Year"].astype(str) + "-" + gdp["Month"].astype(str) + "-" + "01"
# gdp["Date"] = pd.to_datetime(gdp["Date"])

# alternative using month end:
gdp["Month"] = gdp["Quarter"] * 3   # Q1:3, Q2:6, Q3: 9, Q4: 12 
gdp["Year_Month"] = gdp["Year"].astype(str) + "-" + gdp["Month"].astype(str) 
gdp["Date"] = pd.to_datetime(gdp["Year_Month"], format="%Y-%m") + MonthEnd(0)
gdp.index = gdp["Date"] # set index here to enable start and end date prediction range later on

gdp[["Year", "Quarter", "Month", "Date"]].head()

,Year,Quarter,Month,Date
Date,,,,
1947-03-31,1947,1,3,1947-03-31
1947-06-30,1947,2,6,1947-06-30
1947-09-30,1947,3,9,1947-09-30
1947-12-31,1947,4,12,1947-12-31
1948-03-31,1948,1,3,1948-03-31


In [ ]:
px.line(gdp, x="Date", y="Growth Rate", title="GDP Growth Rates")

##### Auto-correlation Function

In [ ]:
print(gdp[gdp["Growth Rate"].isna()])

gdp.dropna(inplace=True)

            Year  Quarter  Real GDP  Growth Rate  Month Year_Month       Date
Date                                                                         
1947-03-31  1947        1  2033.061          NaN      3     1947-3 1947-03-31


In [ ]:
x = gdp["Growth Rate"]

acf_results = acf(x, nlags=12)
print(acf_results)

px.line(y=acf_results, markers=["o"], height=500,
            title=f"Auto-correlation of GDP Growth",
            labels={"x": "Number of Lags", "y":"Auto-correlation"},   
)


[ 1.          0.11766495  0.11756216  0.01360745 -0.03066875 -0.08305177
 -0.00919196 -0.02807425 -0.01067757  0.04546257  0.06213563  0.01987648
 -0.06056621]


##### ARMA Model and Predictions

In [ ]:
# https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMA.html
# https://stats.stackexchange.com/questions/321442/why-is-maximum-likelihood-used-for-arima-instead-of-least-squares
from statsmodels.tsa.arima.model import ARIMA

# gdp.index = gdp["Date"] # set index enables start and end range in arma results prediction later

y = gdp["Growth Rate"]

# order: The (p,d,q) order of the model for the autoregressive, differences, and moving average components. 
# ... d is always an integer, while p and q may either be integers or lists of integers.
model = ARIMA(y, order=(2, 0, 0), missing="drop")
print(type(model))

arma_results = model.fit()
print(type(arma_results))

print(arma_results.summary())

<class 'statsmodels.tsa.arima.model.ARIMA'>
<class 'statsmodels.tsa.arima.model.ARIMAResultsWrapper'>
                               SARIMAX Results                                
Dep. Variable:            Growth Rate   No. Observations:                  294
Model:                 ARIMA(2, 0, 0)   Log Likelihood                 896.528
Date:                Thu, 19 Jan 2023   AIC                          -1785.056
Time:                        03:23:47   BIC                          -1770.322
Sample:                    06-30-1947   HQIC                         -1779.156
                         - 09-30-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0076      0.001      6.899      0.000       0.005       0.010
ar.L1          0.0872      0.

In [ ]:
print(gdp["Date"].min())
print(gdp["Date"].max())

1947-06-30 00:00:00
2020-09-30 00:00:00


In [ ]:
#gdp.tail(8)

In [ ]:
# https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMAResults.predict.html
# start and/or end params must match actual index values in our original data
# ... otherwise we might see KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'

y_pred_example = arma_results.predict(start="2018-12-31", end="2022-12-31")

print(type(y_pred_example))
print(y_pred_example) # looks like this is a series with index corresponding with the original time index


<class 'pandas.core.series.Series'>
2018-12-31    0.007269
2019-03-31    0.006890
2019-06-30    0.006953
2019-09-30    0.007216
2019-12-31    0.006937
2020-03-31    0.007276
2020-06-30    0.005591
2020-09-30   -0.003807
2020-12-31   -0.000647
2021-03-31    0.016459
2021-06-30    0.007189
2021-09-30    0.008846
2021-12-31    0.007655
2022-03-31    0.007789
2022-06-30    0.007630
2022-09-30    0.007635
2022-12-31    0.007612
Freq: Q-DEC, Name: predicted_mean, dtype: float64


In [ ]:
# https://stackoverflow.com/a/40762674/670433 
# ... how to merge dataframe and series that have the same index:
# df.merge(s.to_frame(), left_index=True, right_index=True)

gdp_latest = gdp[gdp["Date"] >= "2018-01-01"]

gdp_with_preds = gdp_latest.merge(y_pred_example, left_index=True, right_index=True,
                    how="outer" # keep all records from both sides even if there is no overlap
                )

gdp_with_preds.rename(columns={"predicted_mean": "Predicted"}, inplace=True)
#gdp_with_preds

In [ ]:
px.line(gdp_with_preds, x="Date", y=["Growth Rate", "Predicted"],
        title="GDP Growth Rates vs Prediction",
        labels={"value": "Quarterly Growth Rate"},
        markers="o"
)

#### Example -- Autoregressive model for Baseball Statistics

In [ ]:
df_nyy = pd.read_excel(baseball_file_url, sheet_name="ny_yankees")

df_nyy.sort_values("Year", ascending=True, inplace=True)

#df_nyy.index = pd.date_range("1903", "2021", freq = "A")
# alternative:
#year_month = df_nyy["Year"].astype(str) + "-12"
#df_nyy.index = pd.to_datetime(year_month, format="%Y-%m") + MonthEnd(0)
# alternative:
df_nyy.index = pd.to_datetime(df_nyy["Year"], format="%Y") + YearEnd(0)

df_nyy.head()

,Year,Tm,Lg,G,W,L,Ties,W-L%
Year,,,,,,,,
1903-12-31,1903,New York Highlanders,AL,136,72,62,2,0.537
1904-12-31,1904,New York Highlanders,AL,155,92,59,4,0.609
1905-12-31,1905,New York Highlanders,AL,152,71,78,3,0.477
1906-12-31,1906,New York Highlanders,AL,155,90,61,4,0.596
1907-12-31,1907,New York Highlanders,AL,152,70,78,4,0.473


In [ ]:
y = df_nyy["W-L%"]

# using 2 lags based on our earlier ACF analysis of the baseball performances
model = ARIMA(y, order=(2, 0, 0))
print(type(model))

arma_results = model.fit()
print(type(arma_results))

print(arma_results.summary())

<class 'statsmodels.tsa.arima.model.ARIMA'>
<class 'statsmodels.tsa.arima.model.ARIMAResultsWrapper'>
                               SARIMAX Results                                
Dep. Variable:                   W-L%   No. Observations:                  118
Model:                 ARIMA(2, 0, 0)   Log Likelihood                 165.908
Date:                Thu, 19 Jan 2023   AIC                           -323.816
Time:                        03:23:48   BIC                           -312.733
Sample:                    12-31-1903   HQIC                          -319.316
                         - 12-31-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5688      0.014     40.322      0.000       0.541       0.596
ar.L1          0.6359      0.

In [ ]:
# https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMAResults.predict.html
# start and/or end params must match actual index values in our original data
# ... otherwise we might see KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'

y_pred_example = arma_results.predict(start="2000-12-31", end="2025-12-31")

print(type(y_pred_example))
#print(y_pred_example.index)
print(y_pred_example) # looks like this is a series with index corresponding with the original time index

<class 'pandas.core.series.Series'>
2000-12-31    0.585678
2001-12-31    0.548829
2002-12-31    0.586108
2003-12-31    0.612913
2004-12-31    0.600021
2005-12-31    0.600791
2006-12-31    0.577264
2007-12-31    0.587205
2008-12-31    0.574535
2009-12-31    0.555683
2010-12-31    0.612407
2011-12-31    0.576675
2012-12-31    0.587205
2013-12-31    0.578350
2014-12-31    0.540151
2015-12-31    0.539097
2016-12-31    0.550814
2017-12-31    0.538554
2018-12-31    0.566711
2019-12-31    0.599737
2020-12-31    0.609329
2021-12-31    0.553784
2022-12-31    0.560083
2023-12-31    0.563917
2024-12-31    0.566070
2025-12-31    0.567266
Freq: A-DEC, Name: predicted_mean, dtype: float64


In [ ]:
nyy_latest = df_nyy[df_nyy["Year"] >= 2000]

nyy_with_preds = nyy_latest.merge(y_pred_example, left_index=True, right_index=True,
                    how="outer" # keep all records from both sides even if there is no overlap
                )

nyy_with_preds.rename(columns={"predicted_mean": "Predicted"}, inplace=True)
#nyy_with_preds

In [ ]:
nyy_with_preds["Date"] = nyy_with_preds.index # makes charting easier to have index as a column
px.line(nyy_with_preds, x="Date", y=["W-L%", "Predicted"],
        title="Baseball Team Performance vs Prediction",
        labels={"value": "Annual Win/Loss Percentage"},
        markers="o"
)

In [ ]:
## https://stackoverflow.com/a/9031848/670433
#import warnings
#warnings.filterwarnings('ignore')
#
#def baseball_performance_prediction_pipeline(df, team_name, n_lags=2): #  , #target="W-L%", #pred_start="2000-12-31", pred_end="2025-12-31"):
#    print("----------")
#    print(team_name.upper())
#    print("N LAGS:", n_lags)
#
#    y = df["W-L%"]
#
#    # MODEL SELECTION
#
#    model = ARIMA(y, order=(n_lags, 0, 0), missing="drop")
#    #print(type(model))
#
#    # MODEL TRAINING
#
#    # https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMAResults.html
#    arma_results = model.fit()
#    #print(type(arma_results))
#
#    # MODEL EVALUATION
#
#    #print(arma_results.summary())
#    print("LOG LIKELIHOOD:", arma_results.llf) # log likelihood
#    
#    # PREDICTIONS
#    # https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMAResults.predict.html
#    # start and/or end params must match actual index values in our original data
#    # ... otherwise we might see KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'
#
#    y_pred_example = arma_results.predict(start="2000-12-31", end="2025-12-31")
#    #print(type(y_pred_example))
#    #print(y_pred_example) # looks like this is a series with index corresponding with the original time index
#
#    # CHARTING
#
#    df_latest = df[df["Year"] >= 2000]
#    df_with_preds = df_latest.merge(y_pred_example, how="outer", # keep all records from both sides even if there is no overlap
#        left_index=True, right_index=True, 
#    )
#    df_with_preds.rename(columns={"predicted_mean": "Predicted"}, inplace=True)
#
#    df_with_preds["Date"] = df_with_preds.index # makes charting easier to have index as a column
#    fig = px.line(df_with_preds, x="Date", y=["W-L%", "Predicted"],
#            title=f"Baseball Team Performance vs Prediction ({team_name}, N Lags = {n_lags})",
#            labels={"value": "Annual Win/Loss Percentage"},
#            markers="o"
#    )
#    fig.show()
#
#    #return df_with_preds
#

# baseball_performance_prediction_pipeline(df_nyy, "NYY", n_lags=1)



#### Example -- Autoregressive Model of Return Volatility

In [ ]:
# earlier we calculated volatility of monthly returns for S&P 500 companies
sp_monthly_volatility.head()

,Year,Month,Volatility
0,1927,12,NaN
1,1928,1,0.007639
2,1928,2,0.006549
3,1928,3,0.007098
4,1928,4,0.009871


In [ ]:
year_month = sp_monthly_volatility["Year"].astype(str) + "-" + sp_monthly_volatility["Month"].astype(str)
sp_monthly_volatility.index = pd.to_datetime(year_month, format="%Y-%m") + MonthEnd(0)
sp_monthly_volatility.head()

,Year,Month,Volatility
1927-12-31,1927,12,NaN
1928-01-31,1928,1,0.007639
1928-02-29,1928,2,0.006549
1928-03-31,1928,3,0.007098
1928-04-30,1928,4,0.009871


In [ ]:
sp_monthly_volatility = sp_monthly_volatility.dropna()

In [ ]:
n_lags = 2

y = sp_monthly_volatility["Volatility"]

model = ARIMA(y, order=(n_lags, 0, 0), missing="drop")
print(type(model))

results = model.fit()
print(type(results))

print(results.summary())

<class 'statsmodels.tsa.arima.model.ARIMA'>
<class 'statsmodels.tsa.arima.model.ARIMAResultsWrapper'>
                               SARIMAX Results                                
Dep. Variable:             Volatility   No. Observations:                 1114
Model:                 ARIMA(2, 0, 0)   Log Likelihood                4354.493
Date:                Thu, 19 Jan 2023   AIC                          -8700.987
Time:                        03:23:50   BIC                          -8680.924
Sample:                    01-31-1928   HQIC                         -8693.401
                         - 10-31-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0098      0.001     10.467      0.000       0.008       0.012
ar.L1          0.6188      0.

In [ ]:
y_pred_example = results.predict(start="2018-12-31", end="2022-12-31")

df_latest = sp_monthly_volatility[sp_monthly_volatility["Year"] >= 2015]

df_with_preds = df_latest.merge(y_pred_example, how="outer", # keep all records from both sides even if there is no overlap
    left_index=True, right_index=True, 
)
df_with_preds.rename(columns={"predicted_mean": "Predicted"}, inplace=True)

df_with_preds["Date"] = df_with_preds.index # makes charting easier to have index as a column
px.line(df_with_preds, x="Date", y=["Volatility", "Predicted"],
        title=f"Monthly Volatility vs Prediction (N Lags = {n_lags})",
        labels={"value": "Volatility"},
        markers="o"
)